In [19]:
import pandas as pd
import numpy as np
import os
import warnings
import random

warnings.filterwarnings('ignore')

In [21]:
path = 'C:/Users/Eyal/Desktop/causal inference/project/tennis/'

matches = pd.read_csv(path + 'Tennis Match Results.csv')
players = pd.read_csv(path + 'atp_players.csv')
D = ['CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
       'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL', 'WPts',
       'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW', 'MaxL',
       'AvgW', 'AvgL']


matches = matches.drop(D, axis=1)
matches = matches[matches['Comment'] == 'Completed']

matches.head()


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,L4,W5,L5,Wsets,Lsets,Comment,Year,WTA,Tier,Gender
0,1.0,Adelaide,Australian Hardcourt Championships,1/3/2000,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,NaN,NaN,NaN,2.0,0.0,Completed,2000,NaN,NaN,ATP
1,1.0,Adelaide,Australian Hardcourt Championships,1/3/2000,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,NaN,NaN,NaN,2.0,0.0,Completed,2000,NaN,NaN,ATP
2,1.0,Adelaide,Australian Hardcourt Championships,1/3/2000,International,Outdoor,Hard,1st Round,3,Escude N.,...,NaN,NaN,NaN,2.0,1.0,Completed,2000,NaN,NaN,ATP
3,1.0,Adelaide,Australian Hardcourt Championships,1/3/2000,International,Outdoor,Hard,1st Round,3,Federer R.,...,NaN,NaN,NaN,2.0,0.0,Completed,2000,NaN,NaN,ATP
4,1.0,Adelaide,Australian Hardcourt Championships,1/3/2000,International,Outdoor,Hard,1st Round,3,Fromberg R.,...,NaN,NaN,NaN,2.0,1.0,Completed,2000,NaN,NaN,ATP


In [22]:
m1 = (matches['W1'].values == 7)
m2 = (matches['L1'].values == 7)
m3 = (matches['W1'].values != 5)
m4 = (matches['L1'].values != 5)

closeSets = matches[(m1 ^ m2) & m3 & m4]
closeSets['W1B'] = np.where(closeSets['W1'] > closeSets['L1'], 1, 0)
closeSets['L1B'] = np.where(closeSets['L1'] > closeSets['W1'], 1, 0)
closeSets['W2B'] = np.where(closeSets['W2'] > closeSets['L2'], 1, 0)
closeSets['L2B'] = np.where(closeSets['L2'] > closeSets['W2'], 1, 0)

In [23]:
df = closeSets.drop(['ATP', 'Location', 'Tournament', 'Series', 'Round',
                     'W3', 'L3', 'W4', 'L4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'WTA', 'Tier'], 
                    axis=1).reset_index()

In [24]:
df.head()

,index,Date,Court,Surface,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,Comment,Year,Gender,W1B,L1B,W2B,L2B
0,2,1/3/2000,Outdoor,Hard,3,Escude N.,Baccanello P.,40.0,655.0,6.0,7.0,7.0,5.0,Completed,2000,ATP,0,1,1,0
1,4,1/3/2000,Outdoor,Hard,3,Fromberg R.,Woodbridge T.,81.0,198.0,7.0,6.0,5.0,7.0,Completed,2000,ATP,1,0,0,1
2,9,1/3/2000,Outdoor,Hard,3,Lisnard J.,Tebbutt M.,134.0,351.0,7.0,6.0,6.0,7.0,Completed,2000,ATP,1,0,0,1
3,16,1/3/2000,Outdoor,Hard,3,Enqvist T.,Federer R.,5.0,65.0,7.0,6.0,6.0,4.0,Completed,2000,ATP,1,0,1,0
4,17,1/3/2000,Outdoor,Hard,3,Escude N.,Dosedel S.,40.0,63.0,7.0,6.0,6.0,4.0,Completed,2000,ATP,1,0,1,0


In [25]:
bothkeys = ['Date', 'Court', 'Surface', 'Best of', 'Comment', 'Year', 'take']

df.head()
random.seed(42)
df['take'] = np.random.rand(len(df))

df1 = df[['Winner', 'W1B', 'W2B', 'WRank', 'LRank'] + bothkeys]
df1 = df1.rename(columns={"Winner": "Player", 'W1B': 'T', 'W2B': 'Y', 'WRank': 'Rank', 'LRank': 'Orank'})
df1 = df1[df1['take'] > 0.5]

df2 = df[['Loser', 'L1B', 'L2B', 'LRank', 'WRank'] + bothkeys]
df2 = df2.rename(columns={"Loser": "Player", 'L1B': 'T', 'L2B': 'Y', 'LRank': 'Rank', 'WRank': 'Orank'})
df2 = df2[df2['take'] < 0.5]

df3 = pd.concat([df1, df2]).reset_index().drop(['take'], axis=1)
df3['diff'] =  df3.apply(lambda row: abs(np.log(row.Rank) - np.log(row.Orank)), axis=1)

df3 = df3[df3['diff'] < 1]
# df3 = df3.drop(['Orank', 'diff'], axis=1) 
df3[['Rank', 'Orank', 'diff']].head(100)

,Rank,Orank,diff
0,81.0,198.0,0.893818
1,40.0,63.0,0.454255
2,128.0,89.0,0.363394
3,121.0,100.0,0.190620
4,83.0,62.0,0.291706
...,...,...,...
174,23.0,21.0,0.090972
175,68.0,128.0,0.632523
177,92.0,110.0,0.178692
180,115.0,88.0,0.267595


In [26]:
players.head()
players['Name'] = players.apply(lambda row: row.full_name + '.', axis=1)

data = pd.merge(df3, players, left_on='Player', right_on='Name')
data = data.drop(['weight_lbs', 'height_ft', 'height_inches', 'birth_day', 'birth_month'], axis=1)

In [27]:
data['Age'] = data.apply(lambda row: row.Year - row.birth_year, axis=1)
data['experience'] =  data.apply(lambda row: row.Year - row.turned_pro, axis=1)
data = data[['Player', 'Rank', 'Court', 'Surface', 'Best of', 'Age', 'weight_kg', 'height_cm', 'experience', 'T', 'Y']].dropna()
#data = pd.get_dummies(data, prefix=['Surface', 'Court', 'Best of'], columns=['Surface', 'Court', 'Best of'])

In [28]:
data.head()

,Player,Rank,Court,Surface,Best of,Age,weight_kg,height_cm,experience,T,Y
0,Fromberg R.,81.0,Outdoor,Hard,3,30.0,88.0,196.0,12.0,1,0
1,Fromberg R.,80.0,Outdoor,Hard,5,30.0,88.0,196.0,12.0,1,1
2,Fromberg R.,71.0,Outdoor,Hard,3,30.0,88.0,196.0,12.0,0,1
3,Fromberg R.,71.0,Indoor,Hard,3,30.0,88.0,196.0,12.0,0,1
4,Fromberg R.,71.0,Outdoor,Hard,3,30.0,88.0,196.0,12.0,1,0


In [29]:
data.to_csv(path + 'data_diff_log.csv', index=False)